In [1]:
%pip install -q -U bitsandbytes
%pip install -q -U git+https://github.com/huggingface/transformers.git
%pip install -q -U git+https://github.com/huggingface/peft.git
%pip install -q -U git+https://github.com/huggingface/accelerate.git
%pip install -q datasets trl
%pip install huggingface_hub

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer

/home/anandrnd/abhishek/Wikisum-main/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model from Hugging Face hub
# base_model = "mistralai/Mistral-7B-Instruct-v0.2"
base_model = "microsoft/phi-2"

# New instruction dataset
dataset_id = "AbhishekJosyula/Wikisun_QA_Format"

# Fine-tuned model
# new_model = "mistral-wikisum-ft"
new_model = "phi2-wikisum-ft"


In [3]:
from datasets import load_dataset

dataset = load_dataset(dataset_id, split='train[:50%]')

In [4]:
dataset = dataset.train_test_split(test_size=0.1)

In [5]:
dataset = dataset.with_format("torch")

In [6]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False
)

In [17]:
device_map = {"": 0}

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map=device_map,
    trust_remote_code=True,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]


In [18]:
model=prepare_model_for_kbit_training(model)

In [19]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [20]:
tokenizer = AutoTokenizer.from_pretrained(base_model, device_map=device_map)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print(tokenizer.model_max_length)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2048


In [21]:
# target_layers = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
target_layers = ['q_proj', 'k_proj', 'v_proj', 'dense', 'fc1', 'fc2', 'lm_head']

peft_params = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_layers
)

In [22]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    logging_steps=1000,
    learning_rate=1e-6,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    evaluation_strategy="epoch"
)

In [23]:
from peft import get_peft_model, prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
peft_model = get_peft_model(model, peft_params)
peft_model.config.use_cache = False

In [24]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset['train'],
    peft_config=peft_params,
    dataset_text_field="Text",
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
    max_seq_length=2048,
    eval_dataset=dataset['test'],
)

trainer.train()

Map: 100%|██████████| 9045/9045 [00:07<00:00, 1169.11 examples/s]
/home/anandrnd/abhishek/Wikisum-main/venv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [25]:
trainer.model.save_pretrained(new_model, push_to_hub=True, repo_id=f"AbhishekJosyula/{new_model}")
trainer.tokenizer.save_pretrained(new_model, push_to_hub=True, repo_id=f"AbhishekJosyula/{new_model}")

/home/anandrnd/abhishek/Wikisum-main/venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:141: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('phi2-wikisum-ft/tokenizer_config.json',
 'phi2-wikisum-ft/special_tokens_map.json',
 'phi2-wikisum-ft/vocab.json',
 'phi2-wikisum-ft/merges.txt',
 'phi2-wikisum-ft/added_tokens.json',
 'phi2-wikisum-ft/tokenizer.json')

In [27]:
testprompt = dataset['test'][0]['Text'].split("[/INST]")

In [28]:
testprompt[1]

' Nicholas Jonathan Anselm Williams (born October 1942 in Walthamstow, Essex, now London, UK), writing as Nicholas Williams or sometimes N.J.A. Williams, is a leading expert on the Cornish language. == Life == While a pupil at Chigwell School, Essex, Williams taught himself Cornish and became a bard of the Cornish Gorseth while still in his teens, taking the bardic name Golvan (\'Sparrow\'). He read classical languages, English language and Celtic in Oxford. After short periods in the universities of Belfast (where he received his PhD) and Liverpool, he was appointed lecturer in Irish in University College Dublin in 1977. In 2006, he was appointed Associate Professor in Celtic Languages there. He married Patricia Smyth from Portadown, County Armagh in 1976. They have three children, Benedict, Jerome, and Dominica who are bilingual in English and Irish. == Work == Williams has written widely on the Celtic languages and their literatures. His works on Irish include the editions The Poems

In [26]:
prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


/home/anandrnd/abhishek/Wikisum-main/venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] Who is Leonardo Da Vinci? [/INST]
[s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [s] [


In [20]:
print(trainer.model.base_model)

LoraModel(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=768, out_features=2304, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=2304, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (c_proj): Linear4bit(in_fea

In [21]:
trainer.push_to_hub("AbhishekJosyula/gpt-wikisum-ft")

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1708195746.7772fafd3cbc.586.0:   0%|          | 0.00/31.0k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AbhishekJosyula/results/commit/e81883940c9b2c4a263606e60b20d01629f71bc8', commit_message='AbhishekJosyula/gpt-wikisum-ft', commit_description='', oid='e81883940c9b2c4a263606e60b20d01629f71bc8', pr_url=None, pr_revision=None, pr_num=None)

In [29]:
testprompt = dataset['test'][7]['Text'].split("[/INST]")
prompt = testprompt[0].split("[INST]")[1]
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


<s>[INST]  You are a helpful writing assistant. Your job is to write a wikipedia style article given the title, summary and information about the topic
    
    title: Iraq national under-17 basketball team

    summary: Iraq national under-17 basketball team<EOT>1

    information: Iraq national under-17 basketball team<EOT>1
      [/INST]
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    



In [32]:
result[0]['generated_text'].split("[/INST]")

['<s>[INST]  Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older brother of Oliver Bjorkstrand.   ',
 "  Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national"]

In [30]:
prompt2="Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt2} [/INST]")
print(result[0]['generated_text'].split("[/INST]"))

["<s>[INST] Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national ", ' ice hockey team. He was drafted by the Los Angeles Kings in the second round, 46th overall, in the 2010 NHL Entry Draft. Bjorkstrand was also selected to play for the Danish national team at the 2011 World Junior Ice Hockey Championships.\n\nOn April 2, 2012, Bjorkstrand signed a two-year entry-level contract with the Los Angeles Kings. He was assigned to the']


In [31]:
result[0]['generated_text'].split("[/INST]")

["<s>[INST] Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national ",
 ' ice hockey team. He was drafted by the Los Angeles Kings in the second round, 46th overall, in the 2010 NHL Entry Draft. Bjorkstrand was also selected to play for the Danish national team at the 2011 World Junior Ice Hockey Championships.\n\nOn April 2, 2012, Bjorkstrand signed a two-year entry-level contract with the Los Angeles Kings. He was assigned to the']